In [25]:
import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings

from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations
from database import SQLDatabase, SQLInsertOperations, SQLQueryOperations


In [26]:
mongo_db = MongoDatabase()
mongo_query_ops = MongoQueryOperations(mongo_db)


In [27]:
query = mongo_query_ops.get_recent_transactions(network="Base", limit=100000)

In [28]:
query[0]


{'block_number': 25068607,
 'transaction_hash': '0xfde1efb93859014f23ff1905fe153cde3a4226c1a171217932ec592951db3272',
 'timestamp': 1736926561,
 'log_data': [{'event': 'Transfer',
   'parameters': {'from': {'value': '0xad4f8f6098232b250f2be2efd02248731246915d',
     'type': 'address',
     'indexed': True,
     'description': None},
    'to': {'value': '0x368ea6f645deb6d1a8692fe93c550d2834150865',
     'type': 'address',
     'indexed': True,
     'description': None},
    'value': {'value': 144976811691545014042624,
     'type': 'uint256',
     'indexed': False,
     'description': None}},
   'contract': '0xAC1Bd2486aAf3B5C0fc3Fd868558b082a531B2B4'},
  {'event': 'Transfer',
   'parameters': {'from': {'value': '0x368ea6f645deb6d1a8692fe93c550d2834150865',
     'type': 'address',
     'indexed': True,
     'description': None},
    'to': {'value': '0xad4f8f6098232b250f2be2efd02248731246915d',
     'type': 'address',
     'indexed': True,
     'description': None},
    'value': {'value':

In [29]:
from web3 import Web3
import web3
import json
from database import SQLDatabase, SQLQueryOperations

sql_db = SQLDatabase()
sql_query_ops = SQLQueryOperations(sql_db)



In [30]:

contract_list = []
# need to check the broken abis
w3 = Web3(web3.HTTPProvider(Settings.BASE_ENDPOINT))
for item in query:
    for log in item['log_data']:
        address = log['contract']

        abi = sql_query_ops.query_evm_contract_abi(network="Base", contract_address=address)
        if abi:
            abi_json = json.loads(abi['abi'])
            contract = w3.eth.contract(address, abi = abi_json)
            contract_list.append(contract)


KeyboardInterrupt: 

In [31]:
len(contract_list)



31555

In [32]:
ERC20_ABI = [
    {
        "constant": True,
        "inputs": [],
        "name": "name",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "symbol",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function"
    }
]

In [33]:
contract_info = []
for i, contract in enumerate(contract_list):
    try:
        token_0 = contract.functions.token0().call()
        token_1 = contract.functions.token1().call()
        token0_contract = w3.eth.contract(address=token_0, abi=ERC20_ABI)
        token1_contract = w3.eth.contract(address=token_1, abi=ERC20_ABI)
        contract_factory = contract.functions.factory().call()
        contract_fee = contract.functions.fee().call()
        contract_token0 = token0_contract.functions.name().call()
        contract_token1 = token1_contract.functions.name().call()
        contract_info.append((
            contract.address,
            contract_factory,
            contract_fee,
            contract_token0,
            contract_token1
        ))
        print(i, contract_factory)
        print(i, contract_fee)
        print(i, contract_token0)
        print(i, contract_token1)
    except Exception as e:
        print(i, e)
        continue

0 ("The function 'token0' was not found in this ", "contract's abi.")
1 ("The function 'token0' was not found in this ", "contract's abi.")
2 ("The function 'fee' was not found in this ", "contract's abi.")
3 ("The function 'fee' was not found in this ", "contract's abi.")
4 ("The function 'token0' was not found in this ", "contract's abi.")
5 ("The function 'token0' was not found in this ", "contract's abi.")
6 ("The function 'fee' was not found in this ", "contract's abi.")
7 ("The function 'token0' was not found in this ", "contract's abi.")
8 ("The function 'token0' was not found in this ", "contract's abi.")
9 ("The function 'token0' was not found in this ", "contract's abi.")
10 ("The function 'fee' was not found in this ", "contract's abi.")
11 ("The function 'token0' was not found in this ", "contract's abi.")
12 ("The function 'token0' was not found in this ", "contract's abi.")
13 ("The function 'fee' was not found in this ", "contract's abi.")
14 ("The function 'fee' was not

KeyboardInterrupt: 

In [34]:
contract_info


[('0x1556465a2F561E69E4Cf97e9272376c06fD8f3AB',
  '0x5e7BB104d84c7CB9B682AaC2F3d509f5F406809A',
  10000,
  'Big Tony',
  'Cod3x USD'),
 ('0xb2cc224c1c9feE385f8ad6a55b4d94E92359DC59',
  '0x5e7BB104d84c7CB9B682AaC2F3d509f5F406809A',
  400,
  'Wrapped Ether',
  'USD Coin'),
 ('0xb2cc224c1c9feE385f8ad6a55b4d94E92359DC59',
  '0x5e7BB104d84c7CB9B682AaC2F3d509f5F406809A',
  400,
  'Wrapped Ether',
  'USD Coin'),
 ('0xb2cc224c1c9feE385f8ad6a55b4d94E92359DC59',
  '0x5e7BB104d84c7CB9B682AaC2F3d509f5F406809A',
  400,
  'Wrapped Ether',
  'USD Coin'),
 ('0x9c087Eb773291e50CF6c6a90ef0F4500e349B903',
  '0x33128a8fC17869897dcE68Ed026d694621f6FDfD',
  500,
  'Virtual Protocol',
  'Wrapped Ether'),
 ('0xC200F21EfE67c7F41B81A854c26F9cdA80593065',
  '0x5e7BB104d84c7CB9B682AaC2F3d509f5F406809A',
  2700,
  'Virtual Protocol',
  'Wrapped Ether'),
 ('0x8deB37B048F4b3c7Bd61EcA7dFcCbef7CbA726dE',
  '0x33128a8fC17869897dcE68Ed026d694621f6FDfD',
  3000,
  'donotfomoew',
  'Wrapped Ether'),
 ('0x0962a51e121aa8371

In [42]:
sql_db = SQLDatabase()
sql_query_ops = SQLQueryOperations(sql_db)

abi = sql_query_ops.query_evm_contract_abi(network="Base", contract_address="0x4200000000000000000000000000000000000006")


In [13]:
import json
abi_json = json.loads(abi['abi'])


In [43]:
addresses = {}
for query in query:
    for log in query['log_data']:
        if log['contract'] not in addresses:
            addresses[log['contract']] = 1
        else:
            addresses[log['contract']] += 1


In [44]:
len(addresses)

3249

In [47]:
sorted_items = sorted(addresses.items(), key=lambda x: x[1], reverse=True)

In [48]:
sorted_items


[('0x4200000000000000000000000000000000000006', 66616),
 ('0xFEC698608Fe71FF77C2B2a8E7fC5253f84B1f8dC', 39860),
 ('0x827922686190790b37229fd06084350E74485b72', 17822),
 ('0xDE6533737cB4a37332b7Bbb233EF5D18d898bCC2', 17531),
 ('0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913', 14609),
 ('0xCF60F60D60aa94bDC2728DBCFC7b2b36BD1079Bc', 8560),
 ('0xffcbF84650cE02DaFE96926B37a0ac5E34932fa5', 8145),
 ('0x0b3e328455c4059EEb9e3f84b5543F74E24e7E1b', 6793),
 ('0x034940f8FB5AEDDaec20731B7FC1A9a55B26B0d2', 5555),
 ('0x940181a94A35A4569E4529A3CDfB74e38FD98631', 5011),
 ('0x0E8a64A06c3f30AF4DDe978afb83D5ae2E834960', 3026),
 ('0xb2cc224c1c9feE385f8ad6a55b4d94E92359DC59', 2905),
 ('0x4F9Fd6Be4a90f2620860d680c0d4d5Fb53d1A825', 2390),
 ('0xcbB7C0000aB88B473b1f5aFd9ef808440eed33Bf', 2319),
 ('0x64ca17E53FB1F3bF0cE5c3a35cad832b0EBb3F1e', 1977),
 ('0xA4f6D3aa28877a94B89DD1bD4087909f6DF229A7', 1766),
 ('0x14Ff9234Da50422aF5bD3ebe8Dc79388eff2180D', 1730),
 ('0xf2614A233c7C3e7f08b1F887Ba133a13f1eb2c55', 1557),
 ('0x

In [111]:
import requests

def get_deployer_basescan(contract_address: str, api_key: str) -> str:
    url = f"https://api.basescan.org/api?module=contract&action=getcontractcreation&contractaddresses={contract_address}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    #print(data['result'])
    if data['status'] == '1' and data['result']:
        return data
    return None

In [112]:
result = get_deployer_basescan(sorted_items[0][0], Settings.BASESCAN_API_KEY)


Maybe decode byte code to 

In [113]:
result

{'status': '1',
 'message': 'OK',
 'result': [{'contractAddress': '0x4200000000000000000000000000000000000006',
   'contractCreator': 'GENESIS',
   'txHash': 'GENESIS_4200000000000000000000000000000000000006',
   'blockNumber': '0',
   'timestamp': '1686789347',
   'contractFactory': '',
   'creationBytecode': '0x6080604052600436106100bc5760003560e01c8063313ce56711610074578063a9059cbb1161004e578063a9059cbb146102cb578063d0e30db0146100bc578063dd62ed3e14610311576100bc565b8063313ce5671461024b57806370a082311461027657806395d89b41146102b6576100bc565b806318160ddd116100a557806318160ddd146101aa57806323b872dd146101d15780632e1a7d4d14610221576100bc565b806306fdde03146100c6578063095ea7b314610150575b6100c4610359565b005b3480156100d257600080fd5b506100db6103a8565b6040805160208082528351818301528351919283929083019185019080838360005b838110156101155781810151838201526020016100fd565b50505050905090810190601f1680156101425780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b34801561015

In [79]:
creators = {}
for item in sorted_items:
    result = get_deployer_basescan(item[0], Settings.BASESCAN_API_KEY)
    if result:
        if result not in creators:
            creators[result] = []
        creators[result].append(item[0])



In [80]:
creators

{'GENESIS': ['...']}

In [81]:
for creator in creators:
    print(creator, len(creators[creator]))


GENESIS 1
0x0b09d65d875127eff2b90511e23fd270beef529d 1
0x51ca29d9828867c363572c37c424e3d6b380c61e 1
0xc54f3080b3cb7cd7ece1c6608b0d87c3a27bf097 1
0x6aaff8af0ae8017725312c388ba3745dfe91185b 1
0xa7a33991e7eb3618141a632de3156c918152496a 1
0xa7356af46c3169310e34d7f1c4dbbf3cb92a0c7e 1
0xe59de1645bb4e4d1bd445914257ff61b7e3323fc 1
0x3885226c26c467b342220132ec9d0b311b2dd43c 1
0x447d07d1a1ed9d25680175a3ff982a15e02603bc 1
0xcf58eac3c7d16796aea617a7e2461c99cacdff1d 1
0xf98cee2bc4848d3e536725c87ec39d9b82e18198 1
0xbc87b3948fecebf93f85f69a9814f3dbb413f2f6 1
0x509bbaf1edb834bc039e1b18dd338ae3b84e28e4 21
0xcc4b7fa65b40387bc8820503f1c82f4502532bd8 1
0x024fbd36161b9c00759fe8b0909ca89b9272aced 10
0x22a1ef11d76ecc4a469490d73521b368262be881 2
0x3f241b9db8f121d36779004a500f8f1e972f511b 1
0x3b76eb1a144e44493a54930fd7956e8e8479692c 1
0xf1c429b0ce94ef9893ef110d2cc100201dce71c8 1
0x9759524cd0d965dcd8fcd9dbfc19d05d23e66e10 1
0x8054a4fbd093808af4529187d3efb9f6301ab92f 1
0x4423b0d6955af39b48cf215577a79ce574299d3f 

In [85]:
creators['0x024fbd36161b9c00759fe8b0909ca89b9272aced']


['0xD5bdD66a87462a608edd28bDec91b88E16409f62',
 '0x1b3a07515800E8b16a46f67A6bab3d4e74148dB0',
 '0x4D9d9913d33Dd7eD6e472f07C6aD4a5BDd9Cc495',
 '0x55B40f8911402568414ec6e7e0d2D87c3D19e65C',
 '0x765bf105ed38D2Ee7801210B4Bb2B8b7D9B3A17f',
 '0xa46179462f33b2c85134e5ec9Cac87073742560a',
 '0x812b7b8e39d4f40e38E091bB2b3DCBED746B9785',
 '0x98Cea1Af4532F05835e72D120ca5DF044121cDeA',
 '0x478ed3dc80b7019787aA1bC49D55916d498c51E1',
 '0x00efd5FCF7EB40D6aF892bDEC9e62bA3bA50e6d9']

In [86]:
from web3 import Web3


Given a Contract address we return the name of the token it trades, ultimately determining what the contract is. From here we just need to be able to determine which event the contract refers to and then we can get the token addresses.


In [88]:
import web3

w3 = web3.Web3(web3.HTTPProvider(Settings.BASE_ENDPOINT))


# maybe decode byte code to get functions of unknown contracts



In [92]:
address = '0xD5bdD66a87462a608edd28bDec91b88E16409f62'

In [93]:
abi = sql_query_ops.query_evm_contract_abi(network="Base", contract_address=address)
abi_json = json.loads(abi['abi'])


In [95]:
contract = w3.eth.contract(address, abi = abi_json)

# maybe decode byte code to get functions of unknown contracts


Look through ABI for key event to determine if it is liquidity contract for DEX

In [98]:
abi_json

[{'inputs': [], 'stateMutability': 'nonpayable', 'type': 'constructor'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'owner',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'int24',
    'name': 'tickLower',
    'type': 'int24'},
   {'indexed': True,
    'internalType': 'int24',
    'name': 'tickUpper',
    'type': 'int24'},
   {'indexed': False,
    'internalType': 'uint128',
    'name': 'amount',
    'type': 'uint128'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'amount0',
    'type': 'uint256'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'amount1',
    'type': 'uint256'}],
  'name': 'Burn',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'owner',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'address',
    'name': 'recipient',
    'type': 'address'},
   {'indexed': True,
    'internalT

Maybe just assume everything is a swap?

In [103]:
# need to be able to determine which function to call, may have to brute force it
token_0 = contract.functions.token0().call()
token_1 = contract.functions.token1().call()


In [109]:
# Determine the factory of the contract this lead to the DEX
contract.functions.factory().call()


'0x0BFbCF9fa4f9C56B0F40a671Ad40E0805A091865'

In [110]:
# determine the fee of the contract
contract.functions.fee().call()


500

In [ ]:
w3.eth.get_transaction_receipt(address)

In [104]:
ERC20_ABI = [
    {
        "constant": True,
        "inputs": [],
        "name": "name",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "symbol",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function"
    }
]
token0_contract = w3.eth.contract(address=token_0, abi=ERC20_ABI)
token1_contract = w3.eth.contract(address=token_1, abi=ERC20_ABI)

In [105]:
token0_contract = w3.eth.contract(address=token_0, abi=ERC20_ABI)
token1_contract = w3.eth.contract(address=token_1, abi=ERC20_ABI)

In [106]:
token0_contract.functions.name().call()


'Virtual Protocol'

In [107]:
token1_contract.functions.name().call()


'Wrapped Ether'

In [ ]:
import requests

def lookup_unknown_selector(selector: str):
    url = f"https://www.4byte.directory/api/v1/signatures/?hex_signature={selector}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            return data['results'][0]['text_signature']
    return None